# Import Library

In [1]:
import numpy as np
import pandas as pd
import random
import math
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn import metrics #akurasi
from sklearn.metrics import confusion_matrix #confussion matrix
from sklearn.metrics import recall_score # recall
from sklearn.metrics import precision_score #precision
from sklearn.metrics import f1_score #f1 score

# Manual Method (Preprocessing and Support)

In [2]:
def count_label(data):
    distinct = [data[0][-1]]
    for i in range(len(data)):
        check = True
        for j in range(len(distinct)):
            if distinct[j] == data[i][-1]:
                check = False
        if check:
            distinct.append(data[i][-1])
    return np.array(distinct).astype(int), len(distinct)

In [3]:
def evaluation(arr_acc):
    mean = np.mean(arr_acc)
    varian = sum((arr_acc - mean) * (arr_acc - mean)) / len(arr_acc)
    std = math.sqrt(varian)
    return mean, std

In [4]:
def handleMissValue(data):
    filled_data = np.array([[0 for i in range(len(data[0]))] for j in range(len(data))])
    cLabel = count_label(data)[1]
    arr_data = [[] for j in range(cLabel)]
    mean = []
    for i in range(len(data)):
        arr_data[int(data[i][-1])-1].append(data[i])
    counter = 0
    for i in range(len(arr_data)):
        mean.append(np.nanmean(arr_data[i], axis=0))
        for j in range(len(arr_data[i])):
            for k in range(len(arr_data[i][j])):
                if np.isnan(arr_data[i][j][k]):
                    arr_data[i][j][k] = mean[i][k]
            filled_data[counter] = arr_data[i][j]
            counter += 1
    return filled_data

In [5]:
def randomize(arr):
    for i in range(len(arr)-1, 0, -1):
        j = random.randint(0, i)
        arr[i], arr[j] = arr[j], arr[i]
    return arr

In [6]:
def normalize(data):
    data = np.array(data).astype(float)
    max_value = np.max(data, axis=0)
    min_value = np.min(data, axis=0)
    for i in range(len(data)):
        for j in range(len(data[0])-1):
            data[i][j] = (data[i][j] - min_value[j]) / (max_value[j] - min_value[j])
    return data

In [7]:
def extraction(file, is_normal = True, is_random = True):
    data = pd.read_excel(file)
    data = handleMissValue(np.array(data))
    data = pd.DataFrame(data)
    data.drop_duplicates()
    data = np.array(data)
    if is_normal:
        data = normalize(data)
    if is_random:
        data = randomize(data.tolist())
    return data

In [8]:
def crossValidation(data, cross_val, cross_index, label_index):
    split_test = int(len(data) * (1 / cross_val))
    change_index = (cross_index - 1) * split_test
    
    for i in range(split_test):
        j = (i + change_index)
        data[i], data[j] = data[j], data[i]
    
    data_train_feature = pd.DataFrame(data[split_test:]).drop([label_index], axis=1)
    data_train_label = pd.DataFrame(pd.DataFrame(data[split_test:]), columns=[label_index]).astype(np.int)
    data_test_feature = pd.DataFrame(data[0:split_test]).drop([label_index], axis=1)
    data_test_label = pd.DataFrame(pd.DataFrame(data[0:split_test]), columns=[label_index]).astype(np.int)
    
    data_train_feature = np.array(data_train_feature)
    data_train_label = np.array(data_train_label)
    data_test_feature = np.array(data_test_feature)
    data_test_label = np.array(data_test_label)

    return data, data_train_feature, data_train_label, data_test_feature, data_test_label

In [9]:
##transform into 1d label array 
def transformArrLabel(X_label, Y_label):
    X_transformed = []
    Y_transformed = []
    for i in range(len(X_label)):
        X_transformed.append(X_label[i][0])
    for i in range(len(Y_label)):
        Y_transformed.append(Y_label[i][0])
    return np.array(X_transformed), np.array(Y_transformed)

In [10]:
def pickCoff(coff):
    choosen_coff = []
    for i in range(len(coff)):
        for j in range(len(coff[i])):
            acc = 0
            for k in range(len(coff[i][j])):
                for l in range(len(coff[i][j][k])):
                    if k == l:
                        acc += coff[i][j][k][l]
            acc /= np.sum(coff[i][j])
            if acc != 1:
                choosen_coff.append(coff[i][j])
                break
            elif j == len(coff[i]) - 1:
                choosen_coff.append(coff[i][j])
    return np.array(choosen_coff)

# Inisialisasi

In [11]:
k = 15
file = 'Dataset Pegawai.xlsx'
cross_val = 10
label_index = 9 #Kinerja
normalisasi = True
random_data = True
data = extraction(file, normalisasi, random_data)

# Testing

In [12]:
k_range = range(1,16)

arr_coff = [[] for i in range(k)]
arr_mean_acc = [0 for i in range(k)]
arr_std_acc  = [0 for i in range(k)]

for k in k_range:
    arr_acc = [0 for i in range(cross_val)]
    for i in range(1, cross_val + 1):
        data, train_f, train_l, test_f, test_l = crossValidation(data, cross_val, i, label_index)
        train_l, test_l = transformArrLabel(train_l, test_l)
        knn = KNeighborsClassifier(n_neighbors = k, algorithm='brute')
        knn.fit(train_f, train_l)
        y_pred = knn.predict(test_f)
        acc = metrics.accuracy_score(test_l, y_pred)
        arr_acc[i-1] = acc
        
        arr_coff[k-1].append(confusion_matrix(test_l, y_pred))

#         print("Cross-"+str(i)+" Acc:"+ str(acc))
#         print("---------------------------------------------------------")

    mean, std = evaluation(arr_acc)
    arr_mean_acc[k-1] = mean
    arr_std_acc[k-1] = std
#     print('K-{0} Mean : {1} with Standar Deviation : {2}.'.format(k,mean,std))
#     print("==========================================================")

In [13]:
print(np.array(arr_mean_acc))
print(np.array(arr_std_acc))
# print(np.array(arr_coff))

[0.95410334 0.94848024 0.95881459 0.95425532 0.95972644 0.95440729
 0.95927052 0.95607903 0.95744681 0.95288754 0.9550152  0.95167173
 0.95547112 0.94954407 0.95319149]
[0.00389248 0.0056231  0.00617141 0.00646388 0.00703862 0.00580698
 0.00525581 0.0056231  0.00633941 0.00641186 0.00781457 0.00960698
 0.00781013 0.00834622 0.0088303 ]


## Manual Testing (Manual vs Library => Precision, Recall, dan F1-Score)

In [14]:
picked_coff = pickCoff(np.array(arr_coff))
# print(np.array(picked_coff))

### Method (Manual)

In [15]:
def calcPrecRec(coff):
    tfn = []
    tfp = []
    ttp = []
    precision = []
    recall = []
    f1score = []
    for i in range(len(coff)):
        tfn.append(np.sum(coff[i], axis = 1))
        tfp.append(np.sum(coff[i], axis = 0))
        ttp.append(np.sum(coff[i], axis = 1))
        for j in range(len(coff[i])):
            for k in range(len(coff[i][j])):
                if j == k:
                    tfn[i][j] -= coff[i][j][k]
                    tfp[i][j] -= coff[i][j][k]
                else:
                    ttp[i][j] -= coff[i][j][k]
    for i in range(len(tfn)):
        precision.append(np.array([0 for i in range(len(tfn[i]))]).astype(np.float32))
        recall.append(np.array([0 for i in range(len(tfn[i]))]).astype(np.float32))
        f1score.append(np.array([0 for i in range(len(tfn[i]))]).astype(np.float32))
        for j in range(len(tfn[i])):
            precision[i][j] = ttp[i][j]/(ttp[i][j] + tfp[i][j])
            recall[i][j] = ttp[i][j]/(ttp[i][j] + tfn[i][j])
            f1score[i][j] = 2 * precision[i][j] * recall[i][j] / (precision[i][j] + recall[i][j])
    return np.array(precision), np.array(recall), np.array(f1score)

In [16]:
def calcAvg(precision, recall, f1score):
    precision_mean = []
    recall_mean = []
    f1score_mean = []
    for i in range(len(precision)):
        precision_mean.append(np.average(precision[i]))
        recall_mean.append(np.average(recall[i]))
        f1score_mean.append(np.average(f1score[i]))
    return np.array(precision_mean), np.array(recall_mean), np.array(f1score_mean)

In [17]:
precision_arr, recall_arr, f1score_arr = calcPrecRec(picked_coff)
precision_arr_avg, recall_arr_avg, f1score_arr_avg = calcAvg(precision_arr, recall_arr, f1score_arr)

### Method (Library)

In [18]:
def splitCoff(picked_coff):
    actual = []
    predict = []
    for i in range(len(picked_coff)):
        predict.append([])
        actual.append([])
        for j in range(len(picked_coff[i])):
            for k in range(len(picked_coff[i][j])):
                for l in range(picked_coff[i][j][k]):
                    if j == k:
                        actual[i].append(j)
                        predict[i].append(j)
                    else:
                        actual[i].append(j)
                        predict[i].append(k)
    return actual, predict

In [19]:
picked_coff = pickCoff(np.array(arr_coff))

precision_lib_arr = []
recall_lib_arr = []
f1score_lib_arr = []

precision_lib_arr_avg = []
recall_lib_arr_avg = []
f1score_lib_arr_avg = []

for i in range(len(picked_coff)):
    actual, predict = splitCoff(picked_coff)
    
    recall_lib = recall_score(actual[i], predict[i], average=None)
    precision_lib = precision_score(actual[i], predict[i], average=None)
    f1score_lib = f1_score(actual[i], predict[i], average=None)
    
    recall_lib_avg = recall_score(actual[i], predict[i], average='macro')
    precision_lib_avg = precision_score(actual[i], predict[i], average='macro')
    f1score_lib_avg = f1_score(actual[i], predict[i], average='macro')
    
    recall_lib_arr.append(recall_lib)
    precision_lib_arr.append(precision_lib)
    f1score_lib_arr.append(f1score_lib)
    
    precision_lib_arr_avg.append(precision_lib_avg)
    recall_lib_arr_avg.append(recall_lib_avg)
    f1score_lib_arr_avg.append(f1score_lib_avg)

### Output (Manual)

#### Precision, Recall and F1-Score (Each Class)

In [20]:
print("-------------Precision--------------")
print(precision_arr)
print()
print("---------------Recall----------------")
print(recall_arr)
print()
print("--------------F1-SCORE---------------")
print(f1score_arr)
print()

-------------Precision--------------
[[0.9314079  0.9531773  0.9878049 ]
 [0.90181816 0.96949154 1.        ]
 [0.9389313  0.971246   1.        ]
 [0.9233871  0.97805643 1.        ]
 [0.9236364  0.9738562  0.97402596]
 [0.936255   0.9591195  0.98876405]
 [0.9404255  0.9738372  1.        ]
 [0.8981818  0.9806452  1.        ]
 [0.9583333  0.971246   1.        ]
 [0.91544116 0.971519   0.9714286 ]
 [0.9403509  0.9862543  0.9878049 ]
 [0.91449815 0.96989965 1.        ]
 [0.9245283  0.9645161  0.9879518 ]
 [0.90118575 0.9807692  1.        ]
 [0.9007092  0.97333336 0.9868421 ]]

---------------Recall----------------
[[0.95555556 0.93442625 0.97590363]
 [0.98804784 0.913738   0.9361702 ]
 [0.972332   0.95       0.9764706 ]
 [0.991342   0.94259816 0.9479167 ]
 [0.9694657  0.9283489  1.        ]
 [0.96311474 0.94720495 0.95652175]
 [0.9650655  0.95988536 0.9875    ]
 [0.988      0.91566265 0.9605263 ]
 [0.97307694 0.96507937 0.97590363]
 [0.9880952  0.92469877 0.9189189 ]
 [0.9925926  0.9409836 

#### Precision, Recall and F1-Score (Average)

In [21]:
print("-------------Precision--------------")
print(precision_arr_avg)
print()
print("---------------Recall----------------")
print(recall_arr_avg)
print()
print("--------------F1-SCORE---------------")
print(f1score_arr_avg)
print()

-------------Precision--------------
[0.9574633  0.95710325 0.9700591  0.9671478  0.9571729  0.9613795
 0.97142094 0.95960903 0.97652644 0.9527962  0.97147006 0.9614659
 0.95899874 0.96065164 0.95362824]

---------------Recall----------------
[0.9552951  0.9459853  0.9662676  0.960619   0.96593815 0.9556138
 0.9708169  0.9547296  0.97135335 0.94390434 0.9698265  0.9546814
 0.9558201  0.96007806 0.9597077 ]

--------------F1-SCORE---------------
[0.9562847  0.9502628  0.9679802  0.96314025 0.9611322  0.9583319
 0.97103596 0.9559529  0.97386885 0.9474523  0.9702237  0.9573757
 0.9571143  0.9593547  0.95587325]



### Output (Library)

#### Precision, Recall and F1-Score (Each Class)

In [22]:
print("-------------Precision--------------")
print(np.array(precision_lib_arr))
print()
print("---------------Recall----------------")
print(np.array(recall_lib_arr))
print()
print("--------------F1-SCORE---------------")
print(np.array(f1score_lib_arr))
print()

-------------Precision--------------
[[0.93140794 0.95317726 0.98780488]
 [0.90181818 0.96949153 1.        ]
 [0.9389313  0.97124601 1.        ]
 [0.9233871  0.97805643 1.        ]
 [0.92363636 0.97385621 0.97402597]
 [0.93625498 0.9591195  0.98876404]
 [0.94042553 0.97383721 1.        ]
 [0.89818182 0.98064516 1.        ]
 [0.95833333 0.97124601 1.        ]
 [0.91544118 0.97151899 0.97142857]
 [0.94035088 0.9862543  0.98780488]
 [0.91449814 0.96989967 1.        ]
 [0.9245283  0.96451613 0.98795181]
 [0.90118577 0.98076923 1.        ]
 [0.90070922 0.97333333 0.98684211]]

---------------Recall----------------
[[0.95555556 0.93442623 0.97590361]
 [0.98804781 0.91373802 0.93617021]
 [0.97233202 0.95       0.97647059]
 [0.99134199 0.94259819 0.94791667]
 [0.96946565 0.92834891 1.        ]
 [0.96311475 0.94720497 0.95652174]
 [0.9650655  0.95988539 0.9875    ]
 [0.988      0.91566265 0.96052632]
 [0.97307692 0.96507937 0.97590361]
 [0.98809524 0.9246988  0.91891892]
 [0.99259259 0.94098361

#### Precision, Recall and F1-Score (Average)

In [23]:
print("-------------Precision--------------")
print(np.array(precision_lib_arr_avg))
print()
print("---------------Recall----------------")
print(np.array(recall_lib_arr_avg))
print()
print("--------------F1-SCORE---------------")
print(np.array(f1score_lib_arr_avg))
print()

-------------Precision--------------
[0.95746336 0.95710324 0.9700591  0.96714784 0.95717285 0.96137951
 0.97142091 0.95960899 0.97652645 0.95279625 0.97147002 0.96146594
 0.95899875 0.96065167 0.95362822]

---------------Recall----------------
[0.95529513 0.94598535 0.96626753 0.96061895 0.96593819 0.95561382
 0.97081696 0.95472966 0.9713533  0.94390432 0.9698266  0.95468135
 0.95582011 0.9600781  0.95970766]

--------------F1-SCORE---------------
[0.95628468 0.95026274 0.96798019 0.96314023 0.9611322  0.95833188
 0.97103596 0.95595288 0.97386887 0.94745233 0.97022373 0.95737571
 0.95711426 0.95935475 0.95587323]

